## Utilisation de LINQ et Z3 pour la résolution de problèmes

### Installation de Z3.Linq

On appelle le package Nuget correspondant. 




In [3]:
#r "nuget: Z3.Linq"

Installed Packages Z3.Linq, 2.0.1

### Exemple court

In [4]:
using Z3.Linq; 

using (var ctx = new Z3Context())
      {
        var theorem = from t in ctx.NewTheorem<Symbols<int, int, int, int, int>>()
              where t.X1 - t.X2 >= 1
              where t.X1 - t.X2 <= 3
              where t.X1 == (2 * t.X3) + t.X5
              where t.X3 == t.X5
              where t.X2 == 6 * t.X4
              select t;
var solution = theorem.Solve();
Console.WriteLine("X1 = {0}, X2 = {1}, X3 = {2}, X4 = {3}, X5 = {4}", solution.X1, solution.X2, solution.X3, solution.X4, solution.X5);

}

X1 = 3, X2 = 0, X3 = 1, X4 = 0, X5 = 1
